Alfred Hoffman (alfredh2) and Aryan Shah (aryans5)

# Part 1: LLM Setup, Prompting, and Embedding Generations

In this part you will learn to setup a Hugging Face LLM, learning to prompt and parse the response, and then finally generate embeddings using an embedding model.
In addition, you will vary the temperature parameter to see how that affects the LLM's response.

**You will need to write functions to**
1. Generate prompts and input them to the tokenizer to the LLM.
2. Decode tokenizer's output by the LLM, and parse the LLM response as a string.


# Task 0: Setting up Environment and the LLM

## Installing necessary packages
First, we install Hugging Face and Llama Index libraries that would be used through out the project.

 **Make sure to run this cell again after a change of runtime type or when a runtime session is terminated.**


In [3]:
# install llama_index
! pip install llama_index==0.10.19 llama_index_core==0.10.19 torch llama-index-embeddings-huggingface peft optimum bitsandbytes

# install auto_gptq
!pip install auto_gptq

# install docx2txt
! pip install docx2txt

INFO: pip is looking at multiple versions of llama-index-embeddings-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of llama-index-program-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-file to determine which version i

## LLM Setup
Run the following cells below to import necessary libraries, setting up the Hugging Face LLM and prompting the LLM to get an response. In this MP we use the Qwen2.5 (千问) LLM from Alibaba Cloud.

In [4]:
# import list of libraries
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings,SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
# instantiate the LLM from the Hugging Face library
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             trust_remote_code=False,
                                             revision="main",
                                             device_map="cuda:0"
                                             )


# instantiate the tokenizer, notice that the model name is the same as LLM
# why do you think is that, feel free to search online.
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Ensure that the above steps complete without any errors. Once successful, the tokenizer and LLM will be loaded and ready to use

**For all steps below, remove ```raise NotImplementedError``` after you have completed the TODO items.**

#  Task 1 - LLM Prompting and Output Response Processing



### Step 1: Prompt Generation

In this step we write a funciton that can generate the prompts used for prompting the LLM. Populate the prompt generating functions below based on the following specs.

**Input:** User query and context

**Output:** Prompt as a string.

**Example:**
1. Context: A customer is having issues with their smartphone battery draining quickly and the phone overheating.
2. User query: My battery is constantly draining, what are some suggestions.
3. Example output prompt:
```
Context: A customer is having issues with their smartphone battery draining quickly and the phone overheating.
Please respond to the following user comment. Use the context above if it is helpful.
User comments: My battery is constantly draining, what are some suggestions.
```
4. Expected format:
```
Context:<context>
Please respond to the following user comment. Use the context above if it is helpful.
User comments: <user query>
```


In [72]:
# TODO 1: Populate the user prompt generating function
def prompt_with_context(context, user_query):
  # change prompt to float, populate context and user comments, add the sentence in between to follow expected format
  prompt = f"""
  Context: {context}
  Please respond to the following user comment. Use the context above if it is helpful.
  User comments: {user_query}
  """
  return prompt

In [74]:
# Task 1 Question 1
# test output with just question
output = prompt_with_context("", "What is the functionality of an LLM?") # no context, question
print(output)

 
  Context:  
  Please respond to the following user comment. Use the context above if it is helpful.
  User comments: What is the functionality of an LLM? 
  


### Step 2: LLM Query Function

In this step we write a function to query the LLM given the context and the user_query, using the prompt generation function that you have implemented above.

Recall that LLM works with tokens instead of strings and characters directly, as such we need to tokenize the prompt first using tokenizer and decode the tokens generated by the LLM.

**Input:** User query and context

**Output:** LLM response as a string

Refer to the [Hugging Face Tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.encode_plus) for additional information.

In [75]:
def get_llm_response(context, user_query, temperature=0.0001):
  # generate the prompt
  prompt = prompt_with_context(context, user_query)

  # setting up prompting messages, at a high-level
  # the overall prompting message sets the role for the LLM (system)
  # in this case LLM answers user query, and then also provide the actual
  # query contents and usesr query.
  # DO NOT change this part!
  messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
  input_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
  )
  # TODO 2: call the tokenizer to tokenize the input_text into tokens
  # you can refer to: https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.encode_plus
  # for additional information of how to use a tokenizer
  # you can also take a look at what texts are tokenized

  # follow expected structure and fill in the necessary parameters

  model_inputs = tokenizer.encode_plus(
      input_text,
      add_special_tokens=True,
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=512
  )

  # change the device for input_ids to cuda0 instead of cpu

  device = model.device
  model_inputs = {key: value.to(device) for key, value in model_inputs.items()}

  # calling the LLM with input tokens to generate tokenized outputs
  generated_token_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    temperature=temperature
  )

  # TODO 3: post process the generated tokens
  # print the generated_token_ids and the model_inputs.input_ids, you will see that
  # the first part of the output: generated_token_ids, returned by the LLM contains
  # also the input tokens. Post processing step should remove input tokens and
  # leaves only the tokens output by the LLM.

  def post_processing(generated_token_ids):
    # TODO 3: insert post processing logic here to remove the input tokens from the
    # generated tokens

    # first, we get the length of the input sequence
    input_len = model_inputs['input_ids'].shape[1]

    # then, we remove the input tokens from the generated tokens
    generated_token_ids = generated_token_ids[:, input_len:]

    return generated_token_ids

  generated_token_ids_post_processed = post_processing(generated_token_ids)

  # TODO 4: decode the generated (post_processed) tokens. Hint: Take a look at
  # the "batch_decode" function of the Hugging Face tokenizer
  # decode the post_processed tokens
  response = tokenizer.batch_decode(generated_token_ids_post_processed, skip_special_tokens=True)[0]

  return response

In [76]:
# Task 1 Question 2
# sample output with just question no context
output = get_llm_response("", "What is the functionality of an LLM?")
print(output)

An LLM (Large Language Model) has various functionalities such as understanding natural language inputs, generating coherent and relevant responses, answering questions, providing information on specific topics, engaging in conversations, learning from interactions with users, improving its performance over time through reinforcement learning or self-supervised training, and many more. These capabilities make LLMs versatile tools for tasks ranging from text generation and summarization to question-answering systems and even creative writing.


### Step 3: Query the LLM with Different Temperature Settings

In this step, we query the LLM using the implemented functions. We are now ready to write the first query to interact with the Qwen 2.5-1.5B LLM.

Additionally, we explore the effect of temperature on LLM inference.

**Example prompt (empty context):** What is the functionality of an LLM?


In [27]:
# TODO 5: uses the get_llm_response function implemented
# and print the LLM's response for the example prompt.
# Keep the temperature parameter at 0.0001 in this step.
response = get_llm_response("", "What is the functionality of an LLM?")
print(response)

An LLM (Large Language Model) has various functionalities such as understanding natural language inputs, generating coherent and relevant responses, answering questions, providing information on specific topics, engaging in conversations, learning from interactions with users, improving its performance over time through reinforcement learning or self-supervised training, and many more. These capabilities make LLMs versatile tools for tasks ranging from text generation and summarization to question-answering systems and even creative writing.


The temperature parameter plays a crucial row in the LLM inference. **Find out what is the role of the temperature parameter and try to generate outputs with different temperature settings. What can you conclude?**

In [35]:
# TODO 6: change the temperature parameter value and see how that affects the LLM output.
# You might want to repeatly generate a few response of the same prompt to see the difference.
# Task 1 Q3, just call the function with a prompt and vary temperature variable
response = get_llm_response("", "What is the functionality of an LLM?", temperature=0.000001)
print(response)

An LLM (Large Language Model) has various functionalities such as understanding natural language inputs, generating coherent and relevant responses, answering questions, providing information on specific topics, engaging in conversations, learning from interactions with users, improving its performance over time through reinforcement learning or self-supervised training, and many more. These capabilities make LLMs versatile tools for tasks ranging from text generation and summarization to question-answering systems and even creative writing.


### Step 4: Hallucination in LLM
This is an open-ended question.

**Construct a prompt such that the prompt leads to a hallucination of LLM. Hallucination have different types: factual hallucination, nonsensical hallucination, or contradicted responses.**

**You can also vary the temperature parameter. Does increasing the temperature help?**

**Does providing some factual context help in reducing the hallucination?**


In [47]:
# TODO 7: Come up a prompt that results in hallucination, then see
# if changing the temperature or providing factual context help?
# Task 1 Q4
# vary context, question, and temperature to hallucinate LLM
output = get_llm_response("Quantum entanglement was not discovered until the 1930s.", "Who wrote the 18th-century poem about quantum entanglement?", temperature = 1)
print(output)

The 18th-century poem you're referring to is "The Entwined Twins" by William Blake.


# Task 2 - Understanding Tokenization and Embeddings
In this part you learn to generate embeddings for texts and understand the relationship between tokenization and embeddings. Generating embeddings is essential for sentence and document understanding as well as building a RAG database, Part 3 of this MP.   

**You need to write functions to**
1. Instantiate an open-sourced Hugging Face embedding model.
2. Encode the given sentence examples.
3. Implement the cosine vector similarity score.
4. Compare the embeddings between those examples and a given reference and select the ones that have a similarity score greater than some threshold.

## Embedding Model Setup
Run the following cells below to setup the embedding model. In this project we use the all-mpnet-base-v2 embedding model. all-mpnet-base-v2 is an embedding model based on Microsoft's mpnet-base models. This model is intented to encode sentences and short paragraphs.

In [50]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from llama_index.core import Document, VectorStoreIndex

In [51]:
# Load a sentence-transformers model for text embeddings from Hugging Face
embed_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Ensure that the above steps complete without any errors. Once successful, embedding model is loaded and ready to go.


**For all steps below, remove ```raise NotImplementedError``` after you have completed the TODO items.**

### Step 1: Embedding Generation

In this step we write a funciton that can generate the embeddings used for downstream tasks such as RAG. Populate the embedding generating functions below based on the following specs.

**Input:** User input text.

**Output:** Text embedding using the embedding model.

In [77]:
# TODO 8: Generate embeddings using the embedding model
def generate_embeddings(text):
  # ensure that input text is converted to numpy
  # encode the text
  embedding = embed_model.encode(text, convert_to_numpy=True)
  return embedding

In [78]:
# Task 2 Q1 and Q2, testing different sentences of different lengths and seeing dimension of output
# dimension of output given by .shape[0]

output = generate_embeddings("What is the functionality of an LLM?")
print(output.shape[0])

output2 = generate_embeddings("Hello.")
print(output2.shape[0])

output3 = generate_embeddings("What is the functionality of an LLM? What are its use cases? What are the benefits? What are the downsides?")
print(output3.shape[0])

768
768
768


### Step 2: Cosine Similarity Score

In this step we write a funciton that ccomputes the similarity of two embeddings. We use cosine similarity.

**Input:** Two embeddings as numpy arrays.

**Output:** The cosine similarity score between the two embeddings.

In [79]:
import numpy as np

# TODO 9: Calculate the cosine similarity score for the two embeddings
def cosine_similarity_score(src_embedding, tar_embedding):

  # calculate dot product
  dot_product = np.dot(src_embedding, tar_embedding)

  # calculate L2 norms for each source and target
  source_norm = np.linalg.norm(src_embedding)
  target_norm = np.linalg.norm(tar_embedding)

  # cosine similarity --> dot product / (L2norm1 * L2norm2)
  score = dot_product / (source_norm * target_norm)

  return score

### Step 3: Comparing Similarity between Sentences.

In this step we uses the implemented embedding generation function and the cosine similarity score function to compute the similiarty between pairs of the provided sentences.

Here we provide a target sentence and a set of 10 other sentences to compare to as part of the sentences.txt file. **Write a program to calculate the cosine similarity score between each sentence in the sentences.txt file and the target sentence.**

**What can you conclude from the similarity scores between the target sentence and each provided sentences? Can you explain what does embedding of a sentence represent?**

In [83]:
# TODO 10: Calculate the cosine similarity score between target sentence and
# each of the other sentences provided in the sentences.txt file.

# mounted sentences.txt in the files tab
# Read the sentences from 'sentences.txt' file
# read-only
with open('sentences.txt', 'r') as file:
    sentences = file.readlines()

# populate the target sentence from the txt file
target_sentence = "The weather today is perfect for a walk in the park."

# call generate_embeddings
target_embedding = generate_embeddings(target_sentence)

# Calculate cosine similarity across the target sentence and each other sentence
similarity_scores = [] # created list to store all the similarity scores
for sentence in sentences:
    # remove whitespace
    sentence = sentence.strip()
    # generate embeddings
    sentence_embedding = generate_embeddings(sentence)
    # calculate similarity score
    similarity_score = cosine_similarity_score(target_embedding, sentence_embedding)
    # append to list of all scores
    similarity_scores.append((sentence, similarity_score))

# printing results, looping thru the list
for sentence, score in similarity_scores:
    print(f"Sentence: {sentence}\nCosine Similarity: {score:}\n")

Sentence: Target Sentence: The weather today is perfect for a walk in the park.
Cosine Similarity: 0.6975216269493103

Sentence: 
Cosine Similarity: 0.029910309240221977

Sentence: Other Sentences:
Cosine Similarity: 0.08279325813055038

Sentence: Jogging in the park is a great way to exercise in the morning.
Cosine Similarity: 0.4796431362628937

Sentence: It might rain later, so the park could get muddy.
Cosine Similarity: 0.45925477147102356

Sentence: Itâ€™s an ideal day to take a walk through the park.
Cosine Similarity: 0.8816159963607788

Sentence: A picnic in the park sounds like a fun idea.
Cosine Similarity: 0.44373199343681335

Sentence: The park is a great place to enjoy todayâ€™s beautiful weather.
Cosine Similarity: 0.704878032207489

Sentence: Todayâ€™s weather makes it wonderful to walk outdoors in the park.
Cosine Similarity: 0.9202273488044739

Sentence: Walking in the park is delightful on a day like today.
Cosine Similarity: 0.8495654463768005

Sentence: The park is